In [ ]:
from datascience import *
import numpy as np

%matplotlib inline
import matplotlib.pyplot as plots
plots.style.use('fivethirtyeight')

# Lecture 11 #

## Grouping review and continuation ##

Previously we've seen how we can use the `tb.group()` method to calculate the number of values in each group. We will now also use to to apply a function separately to each group. 

In [ ]:
all_cones = Table.read_table('cones.csv')
all_cones

In [ ]:
# remove the Color column for now
cones = all_cones.drop('Color').exclude(5)
cones

In [ ]:
# count how many cones there are of each Flavor
cones.group('Flavor')

In [ ]:
# Calculate the minimum price for each Flavor
cones.group('Flavor', min)

In [ ]:
# Get the average price for each Flavor
cones.group('Flavor', np.average)

In [ ]:
# Get a list of prices for each Flavor
cones.group('Flavor', list)

In [ ]:
# User-defined function
def spread(arr):
    return max(arr) - min(arr)

spread(make_array(7, 10, 2))

In [ ]:
# Use your own function in .group
cones.group('Flavor', spread)

## Grouping by multiple columns

We can also group by multiple columns. This will allow give us separate values for each combination of values in the columns we group by. 

In [ ]:
cones

In [ ]:
all_cones

In [ ]:
# Count how many items have a particular Flavor and Color combination
all_cones.group(['Flavor', 'Color'])

In [ ]:
# Take the average price for each particular Flavor and Color combination
all_cones.group(['Flavor', 'Color'], np.average)

## Example - NBA salaries ##

In [ ]:
nba = Table.read_table('nba_salaries.csv').relabeled(3, 'SALARY')
nba

In [ ]:
# Total salary paid by each team, highest first
nba.select('TEAM', 'SALARY').group('TEAM', sum).sort('SALARY sum', descending=True)

In [ ]:
# what happens if we don't select only numerical columns first?
nba.group('TEAM', sum)

In [ ]:
# Average salary paid for each position
nba.select('POSITION', 'SALARY').group('POSITION', np.average)

In [ ]:
# For each team, average salary paid for each position
nba.drop('PLAYER').group(['TEAM', 'POSITION'], np.average)

## Pivot Tables ##

In [ ]:
# read in the cones data again
all_cones = Table.read_table('cones.csv')
all_cones

In [ ]:
# counting how many items in each Flavor-Color category
all_cones.group(['Flavor', 'Color'])

In [ ]:
# Pivoting the data to a Color x Flavor matrix of values
all_cones.pivot('Flavor', 'Color')

In [ ]:
# Pivoting the data to a Flavor x Color matrix of values
all_cones.pivot('Color', 'Flavor')

In [ ]:
# Pivoting so that we have the mean price as values in the Color x Flavor pivot Table
all_cones.pivot('Flavor', 'Color', values='Price', collect=np.average)

In [ ]:
# Pivoting so that we can see all items for each Color x Flavor pivot Table combination
all_cones.pivot('Flavor', 'Color', values='Price', collect=list)

In [ ]:
# Pivoting so that we can see all items for each Flavor x Color pivot Table combination
all_cones.pivot('Color', 'Flavor', values='Price', collect=list)

## Pivot Examples ##

In [ ]:
nba = Table.read_table('nba_salaries.csv').relabeled("'15-'16 SALARY", 'SALARY')
nba

In [ ]:
# For each team, average salary paid for each position
nba.pivot('POSITION', 'TEAM', values='SALARY', collect=np.average)

### Challenge Question

Which NBA teams spent the most on their “starters” in 2015-2016? 

We will assume starting players (i.e., "starters") are the players who are paid the most at each position. 

We will also add the total salaries of starters to the pivot table and sort by total team salary. 

Note: When using the `tb.apply()` method, if no column is specified then the function supplied is applied across all rows in the table. 


In [ ]:
# If we assume the "starter" is the player paid the most in each position, 
# we can use "collect = max"
starters = nba.pivot('POSITION', 'TEAM', values='SALARY', collect=max)

# Now we need to find the total paid for the starters
# NOTE: if no columns are supplied to the tb.apply() method, then the function is applied across rows!
totals = starters.drop("TEAM").apply(sum)

# Add "totals" to our "nba_starters_salaries" table and sort by total
starters.with_column("TOTAL", totals).sort("TOTAL", descending=True)

## Join Examples

In [ ]:
# Make table 1
drinks = Table(['Drink', 'Cafe', 'Price']).with_rows([
    ['Milk Tea', "Willoughby's", 4],
    ['Espresso', 'Blue State Coffee',  2],
    ['Latte',    'Blue State Coffee',  3],
    ['Espresso', 'Koffee?',   2]
])
drinks

In [ ]:
# Make table 2
discounts = Table().with_columns(
    'Coupon % off', make_array(25, 50, 5),
    'Location', make_array("Willoughby's", 'Blue State Coffee', "Willoughby's")
)
discounts

In [ ]:
# Join the tables together
combined = drinks.join('Cafe', discounts, 'Location')
combined

In [ ]:
# We can now do some calculations on the tables
discounted_prices = combined.column('Price') * (1 - combined.column('Coupon % off')/ 100)
combined.with_column('Discounted', discounted_prices)

In [ ]:
drinks

In [ ]:
# Join table with itself
two = drinks.join('Cafe', drinks)
two

#### Join Examples: Tennis

There was recently discussion about whether women tennis players get penalized more than men, for coaching violations.  Let's look at some data.

In [ ]:
# Data on number of code violations at tennis Grand Slam tournaments, 1998-2018
men_violations = Table.read_table('tennis_men_penalties.csv')
women_violations = Table.read_table('tennis_women_penalties.csv')

In [ ]:
men_violations

In [ ]:
women_violations

In [ ]:
combined = men_violations.join('Violation', women_violations)
combined.show()

In [ ]:
combined.barh('Violation')

In [ ]:
men_sets = 460
women_sets = 283
normalized = combined.with_columns(
    'Rate (Men)', combined.column('Count (Men)') / men_sets,
    'Rate (Women)', combined.column('Count (Women)') / women_sets
)
normalized.drop('Count (Men)', 'Count (Women)').barh('Violation')